# 3D Patch Extraction Code

This Python notebook implemented the 3D image patch extraction process used to generate dataset for training deep learning model. 

## Importing Packages

In [ ]:
import SimpleITK as sitk
import numpy as np
from matplotlib import pyplot as plt
from numpy.lib import stride_tricks

## Defining Functions

In [ ]:
# Function used to extract patch from original image data
def cutup(data, blck, strd):
    sh = np.array(data.shape)
    blck = np.asanyarray(blck)
    strd = np.asanyarray(strd)
    nbl = (sh - blck) // strd + 1
    strides = np.r_[data.strides * strd, data.strides]
    dims = np.r_[nbl, blck]
    patch = stride_tricks.as_strided(data, strides=strides, shape=dims)
    return patch

In [ ]:
# Function used to resize image data
def resize_image_itk(itkimage, newSize, resamplemethod=sitk.sitkNearestNeighbor):
    resampler = sitk.ResampleImageFilter()
    originSize = itkimage.GetSize()
    originSpacing = itkimage.GetSpacing()
    newSize = np.array(newSize,float)
    factor = originSize / newSize
    newSpacing = originSpacing * factor
    newSize = newSize.astype(np.int)
    resampler.SetReferenceImage(itkimage)
    resampler.SetSize(newSize.tolist())
    resampler.SetOutputSpacing(newSpacing.tolist())
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))
    resampler.SetInterpolator(resamplemethod)
    itkimgResampled = resampler.Execute(itkimage)
    return itkimgResampled

## Loading Image and Mask Data

In [ ]:
# CT image loading and processing
img_path = 'img_path'
reader = sitk.ImageFileReader()
reader.SetImageIO("GDCMImageIO") # Change to "NrrdImageIO" if the image is in nrrd format
reader.SetFileName(img_path)
image_original = reader.Execute();
img_array_original = sitk.GetArrayFromImage(image_original)/255
img_resized = resize_image_itk(sitk.GetImageFromArray(img_array_original), (256, 256, img_array_original.shape[0]))
image_array = sitk.GetArrayFromImage(img_resized)

print('The size of image is: ', image_array.shape)
print('The range of intensity is from ', np.min(image_array), 'to ', np.max(image_array))

In [ ]:
# Segmentation mask loading and processing
seg_path = 'seg_path'
seg_image = sitk.ReadImage(seg_path)
seg_array_original = sitk.GetArrayFromImage(seg_image)
seg_resized = resize_image_itk(sitk.GetImageFromArray(seg_array_original), (256, 256, seg_array_original.shape[0]))
seg_array = sitk.GetArrayFromImage(seg_resized)
print('The size of image is: ', seg_array.shape)
print('The range of intensity is from ', np.min(seg_array), 'to ', np.max(seg_array))

## Patch Extraction

In [ ]:
# Extracting CT image patches
patches_extracted = cutup(image_array, (64, 128, 128), (32, 64, 64))
patches = np.reshape(patches_extracted, (patches_extracted.shape[0]*patches_extracted.shape[1]*patches_extracted.shape[2],64,128,128))

In [ ]:
# Extracting segmentation mask patches
seg_patches_extracted = cutup(seg_array, (64, 128, 128), (32, 64, 64))
seg_patches = np.reshape(seg_patches_extracted, (seg_patches_extracted.shape[0]*seg_patches_extracted.shape[1]*seg_patches_extracted.shape[2],64,128,128)).astype('int8')

## Writing Generated Patches

In [ ]:
for i in range(patches.shape[0]):
    curr_img = sitk.GetImageFromArray(patches[i,:,:,:])
    sitk.WriteImage(curr_img, 'path/img/patch_%d.nrrd' % i)
    curr_seg = sitk.GetImageFromArray(seg_patches[i,:,:,:])
    sitk.WriteImage(curr_seg, 'path/seg/patch_%d.nrrd' % i)